In [1]:
from pyspark.sql import SparkSession
import mysql.connector

In [2]:
# Spark 세션 시작
spark = SparkSession.builder \
    .appName("CSV to MariaDB") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 06:40:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/24 06:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/24 06:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/24 06:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/09/24 06:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/09/24 06:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [4]:
# CSV 파일 읽기
dong_rent = spark.read.csv("hdfs://master:9000/input/동_임대료.csv", header=True, inferSchema=True)


In [5]:
dong_rent.show()

+------+---------------+-------+-------+-------+
|    구|             동|    1층| 1층 외|   전체|
+------+---------------+-------+-------+-------+
|종로구|         종로구|230,674|160,559|195,617|
|종로구|     청운효자동|188,588|115,520|152,054|
|종로구|         사직동|235,516|132,729|184,123|
|종로구|         삼청동|153,171|   NULL|153,171|
|종로구|         부암동|115,405| 80,929| 98,167|
|종로구|         평창동| 83,479|119,522|101,501|
|종로구|         무악동|   NULL|   NULL|   NULL|
|종로구|         교남동|215,432|152,824|184,128|
|종로구|         가회동|233,527|110,071|171,799|
|종로구|종로1·2·3·4가동|269,591|130,688|200,140|
|종로구|    종로5·6가동|273,313|281,611|277,462|
|종로구|         이화동|178,429|105,291|141,860|
|종로구|         혜화동|159,892| 91,260|125,576|
|종로구|        창신1동|262,631|127,691|195,161|
|종로구|        창신2동|105,489| 76,719| 91,104|
|종로구|        창신3동|214,241|   NULL|214,241|
|종로구|        숭인1동|183,505| 58,548|121,027|
|종로구|        숭인2동|205,581|111,597|158,589|
|  중구|           중구|282,431|256,317|269,374|
|  중구|         소공동|259,429|197,948|228,689|
+

In [6]:
dong_rent.groupBy('구').agg({'1층': 'mean'})

DataFrame[구: string, avg(1층): double]

In [7]:
dong_rent.show()

+------+---------------+-------+-------+-------+
|    구|             동|    1층| 1층 외|   전체|
+------+---------------+-------+-------+-------+
|종로구|         종로구|230,674|160,559|195,617|
|종로구|     청운효자동|188,588|115,520|152,054|
|종로구|         사직동|235,516|132,729|184,123|
|종로구|         삼청동|153,171|   NULL|153,171|
|종로구|         부암동|115,405| 80,929| 98,167|
|종로구|         평창동| 83,479|119,522|101,501|
|종로구|         무악동|   NULL|   NULL|   NULL|
|종로구|         교남동|215,432|152,824|184,128|
|종로구|         가회동|233,527|110,071|171,799|
|종로구|종로1·2·3·4가동|269,591|130,688|200,140|
|종로구|    종로5·6가동|273,313|281,611|277,462|
|종로구|         이화동|178,429|105,291|141,860|
|종로구|         혜화동|159,892| 91,260|125,576|
|종로구|        창신1동|262,631|127,691|195,161|
|종로구|        창신2동|105,489| 76,719| 91,104|
|종로구|        창신3동|214,241|   NULL|214,241|
|종로구|        숭인1동|183,505| 58,548|121,027|
|종로구|        숭인2동|205,581|111,597|158,589|
|  중구|           중구|282,431|256,317|269,374|
|  중구|         소공동|259,429|197,948|228,689|
+

In [8]:
# CSV 파일 읽기
area_pop = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(길단위인구-상권).csv", header=True, inferSchema=True)


In [29]:
# CSV 파일 읽기
area_pop = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(길단위인구-행정동).csv", header=True, inferSchema=True)


In [38]:
# CSV 파일 읽기
area_pop = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(길단위인구-자치구).csv", header=True, inferSchema=True)


In [39]:
area_pop = area_pop.drop('상권_구분_코드', '상권_구분_코드_명')

In [40]:
area_2024 = area_pop.filter(area_pop['기준_년분기_코드'] == 20241)

In [41]:
area_2024.show()

+----------------+-----------+--------------+--------------+----------------+----------------+---------------------+---------------------+---------------------+---------------------+---------------------+--------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|기준_년분기_코드|자치구_코드|자치구_코드_명|총_유동인구_수|남성_유동인구_수|여성_유동인구_수|연령대_10_유동인구_수|연령대_20_유동인구_수|연령대_30_유동인구_수|연령대_40_유동인구_수|연령대_50_유동인구_수|연령대_60_이상_유동인구_수|시간대_00_06_유동인구_수|시간대_06_11_유동인구_수|시간대_11_14_유동인구_수|시간대_14_17_유동인구_수|시간대_17_21_유동인구_수|시간대_21_24_유동인구_수|월요일_유동인구_수|화요일_유동인구_수|수요일_유동인구_수|목요일_유동인구_수|금요일_유동인구_수|토요일_유동인구_수|일요일_유동인구_수|
+----------------+-----------+--------------+--------------+----------------+----------------+---------------------+---------------------+---------------------+--

In [42]:
from pyspark.sql import functions as F

# 필요한 컬럼 선택
area_2019_2024 = area_pop.select('기준_년분기_코드', '자치구_코드', '자치구_코드_명', '총_유동인구_수')

# groupBy와 pivot을 사용해 기준_년분기_코드별 매출 합계 계산
pivot_area = area_2019_2024.groupBy('자치구_코드', '자치구_코드_명') \
                             .pivot('기준_년분기_코드') \
                             .agg(F.sum('총_유동인구_수'))

# 컬럼명을 기준_년분기_코드 -> '20231_매출 금액', '20232_매출 금액' 형식으로 변경
new_column_names = [f'{int(col)}_유동인구' if col.isdigit() else col for col in pivot_area.columns]
pivot_area = pivot_area.toDF(*new_column_names)

# 결과 출력
pivot_area.show()

+-----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
|자치구_코드|자치구_코드_명|20191_유동인구|20192_유동인구|20193_유동인구|20194_유동인구|20201_유동인구|20202_유동인구|20203_유동인구|20204_유동인구|20211_유동인구|20212_유동인구|20213_유동인구|20214_유동인구|20221_유동인구|20222_유동인구|20223_유동인구|20224_유동인구|20231_유동인구|20232_유동인구|20233_유동인구|20234_유동인구|20241_유동인구|
+-----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
|      11590|        동작구|      92500683|      60143936|      88

In [46]:
def ph_area(row):
    sql = """
    INSERT INTO population_history (
    area_id, dong_code, gu_code, 
    populations_20231, populations_20232, populations_20233, populations_20234, 
    populations_20221, populations_20222, populations_20223, populations_20224, 
    populations_20211, populations_20212, populations_20213, populations_20214, 
    populations_20201, populations_20202, populations_20203, populations_20204, 
    populations_20241
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # PySpark DataFrame row에서 값을 추출하여 SQL에 넣기
    cursor.execute(sql, (
         None, 
         None, 
         row['자치구_코드'],
         row['20231_유동인구'], row['20232_유동인구'], row['20233_유동인구'], row['20234_유동인구'], 
         row['20221_유동인구'], row['20222_유동인구'], row['20223_유동인구'], row['20224_유동인구'], 
         row['20211_유동인구'], row['20212_유동인구'], row['20213_유동인구'], row['20214_유동인구'], 
         row['20201_유동인구'], row['20202_유동인구'], row['20203_유동인구'], row['20204_유동인구'], 
         row['20241_유동인구']  # 20241년도 유동인구 추가
    ))

In [47]:
def p_area(row):
           # 삽입할 SQL 쿼리 정의
        sql = """
        INSERT INTO populations (gu_code, dong_code, area_id, total_population, man_population, woman_population, 
                                 10s_population, 20s_population, 30s_population, 40s_population, 50s_population, 60s_population, 
                                 0to6_population, 6to11_population, 11to14_population, 14to17_population, 17to21_population, 21to24_population, 
                                 monday_population, tuesday_population, wendsday_population, thursday_population, friday_population, 
                                 saturday_population, sunday_population)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # 데이터베이스에 데이터를 삽입
        cursor.execute(sql, (
            row['자치구_코드'],        # area_id
            None,      # gu_code
            None,          # dong_code
            row['총_유동인구_수'],    # total_population
            row['남성_유동인구_수'],  # man_population
            row['여성_유동인구_수'],  # woman_population
            row['연령대_10_유동인구_수'],  # 10s_population
            row['연령대_20_유동인구_수'],  # 20s_population
            row['연령대_30_유동인구_수'],  # 30s_population
            row['연령대_40_유동인구_수'],  # 40s_population
            row['연령대_50_유동인구_수'],  # 50s_population
            row['연령대_60_이상_유동인구_수'],  # 60s_population
            row['시간대_00_06_유동인구_수'],    # 0to6_population
            row['시간대_06_11_유동인구_수'],    # 6to11_population
            row['시간대_11_14_유동인구_수'],    # 11to14_population
            row['시간대_14_17_유동인구_수'],    # 14to17_population
            row['시간대_17_21_유동인구_수'],    # 17to21_population
            row['시간대_21_24_유동인구_수'],    # 21to24_population
            row['월요일_유동인구_수'],          # monday_population
            row['화요일_유동인구_수'],          # tuesday_population
            row['수요일_유동인구_수'],          # wednesday_population
            row['목요일_유동인구_수'],          # thursday_population
            row['금요일_유동인구_수'],          # friday_population
            row['토요일_유동인구_수'],          # saturday_population
            row['일요일_유동인구_수']           # sunday_population
        ))

In [48]:
connection = mysql.connector.connect(
    host="13.124.90.34",
    user="root",
    password="enqnrhkwk108",
    database="S11P21D108",
    charset="utf8mb4",  # 문자셋 설정
    collation="utf8mb4_general_ci",  # collation 설정
    autocommit=False,  # Auto-commit을 끄고 트랜잭션 처리
    connection_timeout=28800,  # 타임아웃 증가
)

cursor = connection.cursor()

# DataFrame의 각 행을 MariaDB에 삽입
for row in pivot_area.collect():
    ph_area(row)

# DataFrame의 각 행을 MariaDB에 삽입
for row in area_2024.collect():
    p_area(row)

# 변경사항 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()